# <font color = green>Gesture Recognition -DS C27 batch<font> 

##### <font color = green> By:<font> 
<font color = red>    **1. MOHAMMAD SHAHID RASHID**<font>     <font color = blue>     (mohammad.shahid.rashid@gmail.com) <font> <br> 
<font color = red>    **2. DHARMENDRA BUDHA**<font><font>  <font color = blue>   (dharmendra.budha@gmail.com) <font>
 


## <font color = blue>**Project Overview:**</font>

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

Imagine you are working as a data scientist at a home electronics company which manufactures state of the art smart televisions. You want to develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The training data consists of a few hundred videos categorised into one of the five classes. Each video (typically 2-3 seconds long) is divided into a **sequence of 30 frames(images)**. These videos have been recorded by various people performing one of the five gestures in front of a webcam - similar to what the smart TV will use. 
 
Homogenous data will not be there. It will come from various source of different sizes like 360 * 360 and 120 * 160 pixels. So to utilize them we have to make them uniform in size.


## <font color = blue>**Goals of this Project:**</font>

Build a model to recognise 5 hand gestures. The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

	• Thumbs up:  Increase the volume
	• Thumbs down: Decrease the volume
	• Left swipe: 'Jump' backwards 10 seconds
	• Right swipe: 'Jump' forward 10 seconds  
	• Stop: Pause the movie


We need to accomplish the following in the project:

   1. **Generator**:  The generator should be able to take a batch of videos as input without any error. Steps like **cropping, resizing and normalization** should be performed successfully.
   2. **Model**: Develop a model that is able to train without any errors which will be judged on the total number of parameters (as the inference(prediction) time should be less) and the accuracy achieved.
   3. **Write up**: This should contain the detailed procedure followed in choosing the final model. The write up should start with the reason for choosing the base model, then highlight the reasons and metrics taken into consideration to modify and experiment to arrive at the final model.
   
   
In this project, we are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Import libraries to get started.

`import all required Python libaries`

In [1]:
import numpy as np
import datetime
import os
import cv2

import random as rn
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, ZeroPadding3D
from tensorflow.keras.layers import Conv2D, MaxPooling3D, Conv3D, MaxPooling2D
from tensorflow.keras.layers import LSTM

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
rn.seed(30)
tf.random.set_seed(30)

train_path = './Project_data/train'
val_path = './Project_data/val'
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())

## Generator  Class
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, we are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. We have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
class DataGenerator:
    def __init__(self, width=120, height=120, frames=30, channel=3, 
                 crop = True, normalize = False, affine = False, flip = False, edge = False  ):
        self.width = width   # X dimension of the image
        self.height = height # Y dimesnion of the image
        self.frames = frames # length/depth of the video frames
        self.channel = channel # number of channels in images 3 for color(RGB) and 1 for Gray  
        self.affine = affine # augment data with affine transform of the image
        self.flip = flip
        self.normalize =  normalize
        self.edge = edge # edge detection
        self.crop = crop

    # Helper function to generate a random affine transform on the image
    def __get_random_affine(self): # private method
        dx, dy = np.random.randint(-1.7, 1.8, 2)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        return M

    # Helper function to initialize all the batch image data and labels
    def __init_batch_data(self, batch_size): # private method
        batch_data = np.zeros((batch_size, self.frames, self.width, self.height, self.channel)) 
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        return batch_data, batch_labels

    def __load_batch_images(self, source_path, folder_list, batch_num, batch_size, t): # private method
    
        batch_data,batch_labels = self.__init_batch_data(batch_size)
        # We will also build a agumented batch data
        if self.affine:
            batch_data_aug,batch_labels_aug = self.__init_batch_data(batch_size)
        if self.flip:
            batch_data_flip,batch_labels_flip = self.__init_batch_data(batch_size)

        #create a list of image numbers you want to use for a particular video
        img_idx = [x for x in range(0, self.frames)] 

        for folder in range(batch_size): # iterate over the batch_size
            # read all the images in the folder
            imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
            # Generate a random affine to be used in image transformation for buidling agumented data set
            M = self.__get_random_affine()
            
            #  Iterate over the frames/images of a folder to read them in
            for idx, item in enumerate(img_idx): 
                image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes  
                if self.crop:
                    image = self.__crop(image)
                # If normalize is set normalize the image else use the raw image.
                if self.normalize:
                    resized = self.__normalize(self.__resize(image))
                else:
                    resized = self.__resize(image)
                # If the input is edge detected image then use the sobelx, sobely and laplacian as 3 channel of the edge detected image
                if self.edge:
                    resized = self.__edge(resized)
                
                batch_data[folder,idx] = resized
                if self.affine:
                    batch_data_aug[folder,idx] = self.__affine(resized, M)   
                if self.flip:
                    batch_data_flip[folder,idx] = self.__flip(resized)   

            batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
            
            if self.affine:
                batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
            
            if self.flip:
                if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
                elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        if self.affine:
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
        if self.flip:
            batch_data = np.append(batch_data, batch_data_flip, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_flip, axis = 0) 

        return batch_data, batch_labels
    
    def generator(self, source_path, folder_list, batch_size): # public method
        print( 'Source path = ', source_path, '; batch size =', batch_size)
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(folder_list)//batch_size # calculate the number of batches
            for batch in range(num_batches): # we iterate over the number of batches
                # you yield the batch_data and the batch_labels, remember what does yield do
                yield self.__load_batch_images(source_path, folder_list, batch, batch_size, t) 
            
            # Code for the remaining data points which are left after full batches
            if (len(folder_list) != batch_size*num_batches):
                batch_size = len(folder_list) - (batch_size*num_batches)
                yield self.__load_batch_images(source_path, folder_list, num_batches, batch_size, t)

    # Helper function to perfom affice transform on the image
    def __affine(self, image, M):
        return cv2.warpAffine(image, M, (image.shape[0], image.shape[1]))

    # Helper function to flip the image
    def __flip(self, image):
        return np.flip(image,1)
    
    # Helper function to normalise the data
    def __normalize(self, image):
        return image/127.5-1
    
    # Helper function to resize the image
    def __resize(self, image):
        return cv2.resize(image, (self.width,self.height), interpolation = cv2.INTER_AREA)
    
    # Helper function to crop the image
    def __crop(self, image):
        if image.shape[0] != image.shape[1]:
            return image[0:120, 20:140]
        else:
            return image

    # Helper function for edge detection
    def __edge(self, image):
        edge = np.zeros((image.shape[0], image.shape[1], image.shape[2]))
        edge[:,:,0] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,0],(3,3),0),cv2.CV_64F)
        edge[:,:,1] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,1],(3,3),0),cv2.CV_64F)
        edge[:,:,2] = cv2.Laplacian(cv2.GaussianBlur(image[:,:,2],(3,3),0),cv2.CV_64F)
        return edge

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

## Model
Here we make the model using different functionalities that Keras provides. We use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model.Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

## Model Class

In [5]:
class ModelGenerator(object):
    
    @classmethod
    def c3d1(cls, input_shape, nb_classes):
        """
        Build a 3D convolutional network, based loosely on C3D.
            https://arxiv.org/pdf/1412.0767.pdf
        """
        # Model.
        model = Sequential()
        model.add(Conv3D(
            8, (3,3,3), activation='relu', input_shape=input_shape
        ))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(16, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(Conv3D(32, (3,3,3), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
        model.add(Conv3D(64, (2,2,2), activation='relu'))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

        model.add(Flatten())
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model
    
    @classmethod
    def c3d2(cls, input_shape, nb_classes):
        """
        Build a 3D convolutional network, aka C3D.
            https://arxiv.org/pdf/1412.0767.pdf
        """        
        model = Sequential()
        # 1st layer group
        model.add(Conv3D(16, 3, 3, 3, activation='relu',
                         padding='same', name='conv1',
                         subsample=(1, 1, 1),
                         input_shape=input_shape))
        model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               border_mode='valid', name='pool1'))
        # 2nd layer group
        model.add(Conv3D(32, 3, 3, 3, activation='relu',
                         padding='same', name='conv2',
                         subsample=(1, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool2'))
        # 3rd layer group
        model.add(Conv3D(64, 3, 3, 3, activation='relu',
                         padding='same', name='conv3a',
                         subsample=(1, 1, 1)))
        model.add(Conv3D(64, 3, 3, 3, activation='relu',
                         padding='same', name='conv3b',
                         subsample=(1, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool3'))
        # 4th layer group
        model.add(Conv3D(128, 3, 3, 3, activation='relu',
                         padding='same', name='conv4a',
                         subsample=(1, 1, 1)))
        model.add(Conv3D(128, 3, 3, 3, activation='relu',
                         padding='same', name='conv4b',
                         subsample=(1, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool4'))

        # 5th layer group
        model.add(Conv3D(256, 3, 3, 3, activation='relu',
                         padding='same', name='conv5a',
                         subsample=(1, 1, 1)))
        model.add(Conv3D(256, 3, 3, 3, activation='relu',
                         padding='same', name='conv5b',
                         subsample=(1, 1, 1)))
        model.add(ZeroPadding3D(padding=(0, 1, 1)))
        model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool5'))
        model.add(Flatten())

        # FC layers group
        model.add(Dense(512, activation='relu', name='fc6'))
        model.add(Dropout(0.5))
        model.add(Dense(512, activation='relu', name='fc7'))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model
    
    @classmethod
    def c3d3(cls, input_shape, nb_classes):
        model = Sequential()
        model.add(Conv3D(16, kernel_size=(3, 3, 3), input_shape=input_shape, padding='same'))
        model.add(Activation('relu'))
        model.add(Conv3D(16, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
        model.add(Dropout(0.25))

        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
        model.add(Dropout(0.25))

        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model
 
    @classmethod
    def c3d4(cls, input_shape, nb_classes):
        # Define model
        model = Sequential()

        model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        #Flatten Layers
        model.add(Flatten())

        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))

        #softmax layer
        model.add(Dense(5, activation='softmax'))
        return model
    
    @classmethod
    def c3d5(cls, input_shape, nb_classes):
        # Define model
        model = Sequential()

        model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
        #model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(Dropout(0.25))

        model.add(MaxPooling3D(pool_size=(2,2,2)))

        #Flatten Layers
        model.add(Flatten())

        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.5))

        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))

        #softmax layer
        model.add(Dense(5, activation='softmax'))
        return model   
    
    @classmethod
    def lstm(cls, input_shape, nb_classes):
        """Build a simple LSTM network. We pass the extracted features from
        our CNN to this model predomenently."""
        # Model.
        model = Sequential()
        model.add(LSTM(2048, return_sequences=False,
                       input_shape=input_shape,
                       dropout=0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

    @classmethod
    def lrcn(cls, input_shape, nb_classes):
        """Build a CNN into RNN.
        Starting version from:
            https://github.com/udacity/self-driving-car/blob/master/
                steering-models/community-models/chauffeur/models.py
        Heavily influenced by VGG-16:
            https://arxiv.org/abs/1409.1556
        Also known as an LRCN:
            https://arxiv.org/pdf/1411.4389.pdf
        """
        model = Sequential()

        model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),
            activation='relu', padding='same'), input_shape=input_shape))
        model.add(TimeDistributed(Conv2D(32, (3,3),
            kernel_initializer="he_normal", activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
        model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Flatten()))

        model.add(Dropout(0.5))
        model.add(LSTM(256, return_sequences=False, dropout=0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

    @classmethod
    def mlp(cls, input_shape, nb_classes):
        """Build a simple MLP. It uses extracted features as the input
        because of the otherwise too-high dimensionality."""
        # Model.
        model = Sequential()
        model.add(Flatten(input_shape=input_shape))
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(512))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes, activation='softmax'))

        return model

## Trainer Function

In [6]:
def train(batch_size, num_epochs, model, train_generator, val_generator, optimiser=None):

    curr_dt_time = datetime.datetime.now()

    num_train_sequences = len(train_doc)
    print('# training sequences =', num_train_sequences)
    num_val_sequences = len(val_doc)
    print('# validation sequences =', num_val_sequences)
    print('# batch size =', batch_size)    
    print('# epochs =', num_epochs)

    #optimizer = Adam(lr=rate) 
    #write your optimizer
    if optimiser == None:
        optimiser = Adam() 
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print (model.summary())
    
    model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.mkdir(model_name)
            
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, 
                                 monitor='val_loss', 
                                 verbose=1, 
                                 save_best_only=True,
                                 save_weights_only=True, 
                                 mode='auto', 
                                 period=1)
    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
    callbacks_list = [checkpoint, LR]

    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1

    model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                callbacks=callbacks_list, validation_data=val_generator, 
                validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
    
    K.clear_session()

## Model building

### <font color = red> Model 1 </font> :<font color = blue> Resize to 120*120,  `Raw image input`, `No cropping`, `No normalisation`, `No agumentation`, `No flipped images`, `No edge detection` </font>

In [7]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

batch_size = 20
num_epochs = 20

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)


# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22, 24, 24, 32)    27680     
__________________________________

/mnt/disks/user/anaconda3/envs/cuda101/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
34/34 [==============================] - 114s 3s/step - loss: 73.6653 - categorical_accuracy: 0.2223 - val_loss: 1.6096 - val_categorical_accuracy: 0.1600

Epoch 00001: val_loss improved from inf to 1.60960, saving model to model_init_2021-11-1515_30_45.353161/model-00001-29.13229-0.23228-1.60960-0.16000.h5
Epoch 2/20
34/34 [==============================] - 16s 467ms/step - loss: 1.6234 - categorical_accuracy: 0.1916 - val_loss: 1.5651 - val_categorical_accuracy: 0.2900

Epoch 00002: val_loss improved from 1.60960 to 1.56513, saving model to model_init_2021-11-1515_30_45.353161/model-00002-1.61635-0.23529-1.56513-0.29000.h5
Epoch 3/20
34/34 [==============================] - 14s 433ms/step - loss: 1.5785 - categorical_accuracy: 0.3326 - val_loss: 1.5509 - val_categorical_accuracy: 0.2400

Epoch 00003: val_loss improved from 1.56513 to 1.55091, saving model to model_init_2021-11-1515_30_45.353161/model-00003-1.59481-0.30392-1.55091-0.24000.h5
Epoch 4/20
34/34 [==============


### <font color = red> Model 2 </font> :<font color = blue> Resize to 120*120, `agumentation`, `No flipped images`, `No cropping`, `No normalisation`, `No edge detection`</font>

In [8]:
train_gen = DataGenerator(affine=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22, 24, 24, 32)    27680     
__________________________________

34/34 [==============================] - 17s 516ms/step - loss: 0.8475 - categorical_accuracy: 0.6564 - val_loss: 1.0181 - val_categorical_accuracy: 0.6100

Epoch 00020: val_loss did not improve from 0.89991


### <font color = red> Model 3 </font> :<font color = blue> Resize to 120*120, `agumentation`, `flipped images`, `No normalisation`, `No cropping`, `No edge detection`</font>

In [9]:
train_gen = DataGenerator(affine=True, flip=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22, 24, 24, 32)    27680     
__________________________________


Epoch 00019: val_loss did not improve from 0.75279
Epoch 20/20
34/34 [==============================] - 21s 621ms/step - loss: 0.5433 - categorical_accuracy: 0.7960 - val_loss: 0.6684 - val_categorical_accuracy: 0.7500

Epoch 00020: val_loss improved from 0.75279 to 0.66837, saving model to model_init_2021-11-1515_44_17.698701/model-00020-0.66264-0.74837-0.66837-0.75000.h5


### <font color = red> Model 4 </font> :<font color = blue> Resize to 120*120, `agumentation`, `flipped images`, `normalisation`, `No cropping`, `No edge detection`</font>

In [10]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22, 24, 24, 32)    27680     
__________________________________

### <font color = red> Model 5 </font> :<font color = blue> Resize to 120*120, `agumentation`, `flipped images`, `normalisation`, `cropping`, `No edge detection`</font>

In [11]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22, 24, 24, 32)    27680     
__________________________________

### <font color = red> Model 6 </font> :<font color = blue> Resize to 120*120, `agumentation`, `flipped images`, `normalisation`, `cropping`, `edge detection`</font>

In [12]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d1(input_shape, num_classes)


train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 22, 24, 24, 32)    27680     
__________________________________

34/34 [==============================] - 22s 667ms/step - loss: 1.6102 - categorical_accuracy: 0.2355 - val_loss: 20.4402 - val_categorical_accuracy: 0.1600

Epoch 00019: val_loss did not improve from 19.40549
Epoch 20/20
34/34 [==============================] - 22s 663ms/step - loss: 1.6087 - categorical_accuracy: 0.2060 - val_loss: 21.4176 - val_categorical_accuracy: 0.1800

Epoch 00020: val_loss did not improve from 19.40549

Epoch 00020: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.



### <font color = red> Model 7 </font> :<font color = blue> Resize to 120*120, `All except normalization`</font>

In [14]:
train_gen = DataGenerator(affine=True, flip=True, normalize=False, crop=True, edge=False)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d5(input_shape, num_classes)


train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0         
________________________________


Epoch 00016: val_loss improved from 1.48926 to 1.47001, saving model to model_init_2021-11-1516_19_28.358405/model-00016-1.50791-0.30065-1.47001-0.41000.h5
Epoch 17/20
34/34 [==============================] - 21s 632ms/step - loss: 1.5718 - categorical_accuracy: 0.2994 - val_loss: 1.4912 - val_categorical_accuracy: 0.4500

Epoch 00017: val_loss did not improve from 1.47001
Epoch 18/20
34/34 [==============================] - 21s 623ms/step - loss: 1.5142 - categorical_accuracy: 0.3157 - val_loss: 1.5342 - val_categorical_accuracy: 0.3000

Epoch 00018: val_loss did not improve from 1.47001

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 19/20
34/34 [==============================] - 20s 615ms/step - loss: 1.4405 - categorical_accuracy: 0.3289 - val_loss: 1.4700 - val_categorical_accuracy: 0.4400

Epoch 00019: val_loss improved from 1.47001 to 1.47000, saving model to model_init_2021-11-1516_19_28.358405/model-00019-1.45983-0.31699-1.47000-0.44000.

### <font color = red> Model 8 </font> :<font color = blue> Resize to 120*120, `No agumentation,No flipped images,No normalisation,No cropping,No edge detection`</font>

In [16]:
train_gen = DataGenerator()
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (30,120,120, 3)
num_classes = 5

model = model_gen.c3d3(input_shape, num_classes)

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 16)  0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 40, 40, 16)    0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 40, 40, 16)    0         
________________________________

34/34 [==============================] - 16s 472ms/step - loss: 1.6722 - categorical_accuracy: 0.4019 - val_loss: 1.9808 - val_categorical_accuracy: 0.2800

Epoch 00017: val_loss did not improve from 1.60063
Epoch 18/20
34/34 [==============================] - 16s 467ms/step - loss: 1.8805 - categorical_accuracy: 0.3454 - val_loss: 2.1531 - val_categorical_accuracy: 0.2300

Epoch 00018: val_loss did not improve from 1.60063

Epoch 00018: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 19/20
34/34 [==============================] - 16s 482ms/step - loss: 1.7570 - categorical_accuracy: 0.3825 - val_loss: 1.8274 - val_categorical_accuracy: 0.3400

Epoch 00019: val_loss did not improve from 1.60063
Epoch 20/20
34/34 [==============================] - 16s 467ms/step - loss: 1.9354 - categorical_accuracy: 0.2920 - val_loss: 1.7598 - val_categorical_accuracy: 0.3100

Epoch 00020: val_loss did not improve from 1.60063

Epoch 00020: ReduceLROnPlateau reducing learning 

### <font color = red> Model 9 </font> :<font color = blue> Resize to 120*120, `agumentation, flipped images, normalisation, cropping, edge detection`</font>

In [24]:
train_gen = DataGenerator(affine=True, flip=True, normalize=True, crop=True, edge=True)
val_gen = DataGenerator()
model_gen = ModelGenerator()

input_shape = (20,120,120, 3)
num_classes = 5

model = model_gen.c3d4(input_shape, num_classes)

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = val_gen.generator(val_path, val_doc, batch_size)
train(batch_size, num_epochs, model, train_generator, val_generator)

# training sequences = 663
# validation sequences = 100
# batch size = 20
# epochs = 20
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 20, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 20, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 20, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 60, 60, 16)    64        
__________________________________

34/34 [==============================] - 22s 660ms/step - loss: 1.4277 - categorical_accuracy: 0.4343 - val_loss: 587.3335 - val_categorical_accuracy: 0.3300

Epoch 00018: val_loss did not improve from 34.87786

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 19/20
34/34 [==============================] - 22s 657ms/step - loss: 1.4008 - categorical_accuracy: 0.4078 - val_loss: 783.2123 - val_categorical_accuracy: 0.3100

Epoch 00019: val_loss did not improve from 34.87786
Epoch 20/20
34/34 [==============================] - 21s 638ms/step - loss: 1.4964 - categorical_accuracy: 0.3728 - val_loss: 775.3421 - val_categorical_accuracy: 0.2400

Epoch 00020: val_loss did not improve from 34.87786

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1.9531250927684596e-06.


# <font color = blue> Observation: <font> 
`Model 3` is having training and validation accuracy as `79.6%` and `75.0%` which shows model is Good model and able to learn the behaviour. 

===================================================================================

# <font color = green> Conclusion:<font> 

After all above model experiment, we are going to choose `Model 3` - <font color = blue> Resize to 120*120, `agumentation`, `flipped images`, `No normalisation`, `No cropping`, `No edge detection`</font> as good model which performed well.

`Epoch 00020: val_loss improved from 0.75279 to 0.66837, saving model to model_init_2021-11-1515_44_17.698701/model-00020-0.66264-0.74837-0.66837-0.75000.h5`

Number of Epoch = 20         
Batch Size = 20         
Number of parameters =  16,612,069

Best Model file: `model-00020-0.66264-0.74837-0.66837-0.75000.h5`

## <font color="green">========================================================================== </font>
## <center> <font color="blue">                       End of the case study , thank you                     </font> </center>
## <font color="green">========================================================================== </font>